In [1]:
import pandas as pd
from osgeo import gdal
import math
import os
import glob
from PIL import Image # PIL library supports only TIF format but not newer TIFF
import numpy as np
from skimage import util, exposure

In [2]:
data_directory = '../data/'
asf_data_directory = data_directory + '3_processed_ASF_data/'
interpolated_AIS_data_directory = data_directory + '5_interpolated_AIS_data/'
#annotated_asf_data_directory = data_directory + '6_annotated_ASF_data/'
cutted_asf_data_directory = data_directory + '6_cutted_ASF_data/'
#sliced_asf_data_directory = data_directory +'7_sliced_ASF_data/'
results_directory = '../results/'

In [9]:
newest_results_file = "2023-01-13_00-58-29_full_data_df.csv"
full_data_df = pd.read_csv(results_directory + newest_results_file, index_col=0)
full_data_df

,asf_file,image_datetime,date,minLON,minLAT,maxLON,maxLAT,widthpx,heightpx,datetime_lower,datetime_upper,ais_file
0,subset_0_of_S1A_IW_GRDH_1SDV_20210106T015004_2...,2021-01-06 01:50:18.500,2021-01-06,-118.417484,33.310909,-117.675835,33.792945,8256,5366,2021-01-06 01:40:18.500,2021-01-06 02:00:18.500,AIS_2021_01_06.csv
1,subset_1_of_S1A_IW_GRDH_1SDV_20210106T135303_2...,2021-01-06 13:53:15.500,2021-01-06,-118.358644,31.787350,-116.685352,32.904135,18627,12432,2021-01-06 13:43:15.500,2021-01-06 14:03:15.500,AIS_2021_01_06.csv
2,subset_10_of_S1A_IW_GRDH_1SDV_20210307T015003_...,2021-03-07 01:50:17.500,2021-03-07,-118.500855,33.294096,-117.759475,33.776401,8253,5369,2021-03-07 01:40:17.500,2021-03-07 02:00:17.500,AIS_2021_03_07.csv
3,subset_11_of_S1B_IW_GRDH_1SDV_20210306T015804_...,2021-03-06 01:58:16.500,2021-03-06,-119.682261,34.259304,-119.487956,34.384619,2163,1395,2021-03-06 01:48:16.500,2021-03-06 02:08:16.500,AIS_2021_03_06.csv
4,subset_5_of_S1B_IW_GRDH_1SDV_20210306T015735_2...,2021-03-06 01:57:49.500,2021-03-06,-118.373353,33.409416,-117.973153,33.742602,4455,3709,2021-03-06 01:47:49.500,2021-03-06 02:07:49.500,AIS_2021_03_06.csv
5,subset_12_of_S1B_IW_GRDH_1SDV_20210406T014917_...,2021-04-06 01:49:29.500,2021-04-06,-117.504909,32.430804,-117.128335,32.677391,4192,2745,2021-04-06 01:39:29.500,2021-04-06 01:59:29.500,AIS_2021_04_06.csv
6,subset_13_of_S1B_IW_GRDH_1SDV_20210505T015737_...,2021-05-05 01:57:51.500,2021-05-05,-118.369253,33.310377,-117.952344,33.737795,4641,4758,2021-05-05 01:47:51.500,2021-05-05 02:07:51.500,AIS_2021_05_05.csv
7,subset_14_of_S1B_IW_GRDH_1SDV_20210505T015806_...,2021-05-05 01:58:18.500,2021-05-05,-120.182454,34.110307,-119.868223,34.312787,3498,2254,2021-05-05 01:48:18.500,2021-05-05 02:08:18.500,AIS_2021_05_05.csv
8,subset_15_of_S1B_IW_GRDH_1SDV_20210605T014920_...,2021-06-05 01:49:32.500,2021-06-05,-117.740009,32.063002,-117.092324,32.426101,7210,4042,2021-06-05 01:39:32.500,2021-06-05 01:59:32.500,AIS_2021_06_05.csv
9,subset_16_of_S1B_IW_GRDH_1SDV_20210605T135153_...,2021-06-05 13:52:05.500,2021-06-05,-118.573557,33.336484,-117.917068,33.761297,7308,4729,2021-06-05 13:42:05.500,2021-06-05 14:02:05.500,AIS_2021_06_05.csv


In [10]:
full_data_df = full_data_df.iloc[2:3]
full_data_df

,asf_file,image_datetime,date,minLON,minLAT,maxLON,maxLAT,widthpx,heightpx,datetime_lower,datetime_upper,ais_file
2,subset_10_of_S1A_IW_GRDH_1SDV_20210307T015003_...,2021-03-07 01:50:17.500,2021-03-07,-118.500855,33.294096,-117.759475,33.776401,8253,5369,2021-03-07 01:40:17.500,2021-03-07 02:00:17.500,AIS_2021_03_07.csv


In [5]:
def calculate_px_from_lon2(row, xOrigin, pixelWidth):  
    return round((row['LON'] - xOrigin) / pixelWidth)

In [6]:
def calculate_px_from_lat2(row, yOrigin, pixelHeight):  
    return round((yOrigin - row['LAT']) / pixelHeight)

In [12]:
SHIP_WIDTH_PX = 50 #50
SHIP_HEIGHT_PX = 50 #50
OBJECT_CLASS = 0
FILES_FORMAT_TO_PROCESS = '.png'
TAG_MAIN_PICTURE = False #takes a lot of time - tag not tiled big picture

for index, row in full_data_df.iterrows():
    print(f"File: {index}")
    asf_file_name = row['asf_file'].strip(".tif")
    print(f"ASF file name: {asf_file_name}")
    ais_file_name = 'interpolated_processed_' + row['ais_file']
    print(f"AIS file name: {ais_file_name}")
    ais_file_df = pd.read_csv(interpolated_AIS_data_directory + ais_file_name, index_col=0)
    # HERE STARTS THE DIFFERENCE
    directory = cutted_asf_data_directory + asf_file_name
    print(f'directory: {directory}')
    if os.path.exists(directory):
        pictures=list()
        for file in os.listdir(directory):
            if file.endswith(FILES_FORMAT_TO_PROCESS):
                pictures.append(file)
        if TAG_MAIN_PICTURE == False:
            pictures.remove('original_picture.png')
            #print(pictures)
        print(pictures)
        for picture in pictures:
            print(f'picture: {picture}')
            picture_name = picture.strip(FILES_FORMAT_TO_PROCESS)
            print(f'picture_name: {picture_name}')
            in_ds = gdal.Open(directory + '/' + picture)
            #print(in_ds)
            image_width = in_ds.RasterXSize
            image_height = in_ds.RasterYSize
            geo_transform = in_ds.GetGeoTransform()
            
            minx = geo_transform[0] #minLON
            miny = geo_transform[3] + image_width*geo_transform[4] + image_height*geo_transform[5] #minLAT
            maxx = geo_transform[0] + image_width*geo_transform[1] + image_height*geo_transform[2] #maxLON
            maxy = geo_transform[3] #maxLAT
            
            xOrigin = geo_transform[0] # NEEDED
            yOrigin = geo_transform[3] # NEEDED
            pixelWidth = geo_transform[1] # NEEDED
            pixelHeight = -geo_transform[5] # NEEDED
            print(f"image_width: {image_width}, image_height: {image_height}")
            #print(f"pixel_width: {pixelWidth}, pixel_height: {pixelHeight}")
            print(f"xOrigin: {xOrigin}, yOrigin: {yOrigin}")
            #display(ais_file_df.head())
            
            # Restrict AIS data by the given conditions
            # condition for latitude: between 19.193 and 21.1203
            condition1 = (ais_file_df.LAT > miny) & (ais_file_df.LAT < maxy)
            # condition for latitude: between -157.0894 and -154.4233
            condition2 = (ais_file_df.LON > minx) & (ais_file_df.LON < maxx)
            # final dataframe
            target_area = ais_file_df[condition1 & condition2].copy().reset_index(drop=True)
            #display(target_area)
            # Further work
            if len(target_area) != 0:
                print('##############')
                print('SHIPS DETECTED')
                print('##############')
                # TODO: if to save empty txt file?
                # YOLO format do not need empty files for pictures
                for index, row in target_area.iterrows():
                    target_area['LONpx_X2'] = target_area.apply(lambda row: calculate_px_from_lon2(row, xOrigin, pixelWidth), axis=1)
                    target_area['LATpx_Y2'] = target_area.apply(lambda row: calculate_px_from_lat2(row, yOrigin, pixelHeight), axis=1)
                
                display(target_area.head())
                print(f"Number of ships founded (including black areas = 0 pixel value): {len(target_area)}")
                    
                current_file = directory + '/' + picture_name + '.txt'
                if os.path.exists(current_file):
                    os.remove(current_file) #remove old files
                    
                ships_counter = 0
                for index, row in target_area.iterrows():
                    
#                     # Check pixel value - if (0,0,0) then exclude it from the dataset
#                     im = Image.open(directory + '/' + picture) # Can be many different formats.
#                     pix = im.load()
#                     print(f"Image size: {im.size}")  # Get the width and hight of the image for iterating over
#                     try: 
#                         pixels = pix[row['LONpx_X2'], row['LATpx_Y2']]
#                         print(f"Image pixels values: {pixels}")
#                         #print(pixels[0], pixels[1], pixels[2])
#                         if pixels[0]==0 and pixels[1]==0 and pixels[2]==0:
#                             # DO NOT TAG THE DATA THEN - it is black area
#                             print("0 pixels values - not tagging")
#                             continue
#                         ships_counter += 1  
#                     except:
#                         print("IndexError: image index out of range")
#                         continue
                    
                    file_object = open(current_file, 'a')
                    #OBJECT_CLASS, SHIP_WIDTH_PX, SHIP_HEIGHT_PX
                    #print("before normalization")
                    #print("LONpx_X2", row['LONpx_X2'])
                    #print("LATpx_Y2", row['LATpx_Y2'])
                    X_CENTER_AXIS_VALUE = row['LONpx_X2']
                    Y_CENTER_AXIS_VALUE = row['LATpx_Y2']
                    #print(OBJECT_CLASS, X_CENTER_AXIS_VALUE, Y_CENTER_AXIS_VALUE, SHIP_WIDTH_PX, SHIP_HEIGHT_PX)
                    #print("after normalization")
                    #X_CENTER_AXIS_VALUE = (X_CENTER_AXIS_VALUE + SHIP_WIDTH_PX/2) / image_width
                    #Y_CENTER_AXIS_VALUE = (Y_CENTER_AXIS_VALUE + SHIP_HEIGHT_PX/2) / image_height
                    X_CENTER_AXIS_VALUE = X_CENTER_AXIS_VALUE / image_width
                    Y_CENTER_AXIS_VALUE = Y_CENTER_AXIS_VALUE / image_height
                    SHIP_WIDTH_normalized = SHIP_WIDTH_PX / image_width
                    SHIP_HEIGHT_normalized = SHIP_HEIGHT_PX / image_height
                    #print(OBJECT_CLASS, X_CENTER_AXIS_VALUE, Y_CENTER_AXIS_VALUE, SHIP_WIDTH_normalized, SHIP_HEIGHT_normalized)
                    string_to_write = f"{OBJECT_CLASS} {X_CENTER_AXIS_VALUE} {Y_CENTER_AXIS_VALUE} {SHIP_WIDTH_normalized} {SHIP_HEIGHT_normalized}\n"
                    file_object.write(string_to_write)
                    # Close the file       
                    file_object.close()
                print(f"Number of ships founded (including black areas = 0 pixel value): {len(target_area)}")
                print(f"Number of ships founded after excluding black areas == 0 pixel value: {ships_counter}")
            else:
                print()
                #print('Sorry, no ships on the selected area')
                #file_object = open(directory + '/' + picture_name + '.txt', 'a')
                #file_object.close()
            print()
    else:
        print('ERROR!! no picture data')
        print('TRY ANNOTATION AGAIN')
        continue

File: 2
ASF file name: subset_10_of_S1A_IW_GRDH_1SDV_20210307T015003_20210307T015032_036886_045698_BCE0_TC
AIS file name: interpolated_processed_AIS_2021_03_07.csv
directory: ../data/6_cutted_ASF_data/subset_10_of_S1A_IW_GRDH_1SDV_20210307T015003_20210307T015032_036886_045698_BCE0_TC
['subset1.png', 'subset10.png', 'subset100.png', 'subset101.png', 'subset102.png', 'subset103.png', 'subset104.png', 'subset105.png', 'subset106.png', 'subset107.png', 'subset108.png', 'subset109.png', 'subset11.png', 'subset110.png', 'subset111.png', 'subset112.png', 'subset113.png', 'subset114.png', 'subset115.png', 'subset116.png', 'subset117.png', 'subset12.png', 'subset13.png', 'subset14.png', 'subset15.png', 'subset16.png', 'subset17.png', 'subset18.png', 'subset19.png', 'subset2.png', 'subset20.png', 'subset21.png', 'subset22.png', 'subset23.png', 'subset24.png', 'subset25.png', 'subset26.png', 'subset27.png', 'subset28.png', 'subset29.png', 'subset3.png', 'subset30.png', 'subset31.png', 'subset32.p

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass,LONpx_X2,LATpx_Y2
0,366943960,2021-03-07 01:50:17.500,33.42467,-117.78567,29.5,76.6,511.0,CATALINA EXPRESS,IMO8967888,WCJ6210,60.0,0.0,30.0,6.0,NaN,NaN,A,281,75


Number of ships founded (including black areas = 0 pixel value): 1
Number of ships founded (including black areas = 0 pixel value): 1
Number of ships founded after excluding black areas == 0 pixel value: 0

picture: subset116.png
picture_name: subset116
image_width: 640, image_height: 640
xOrigin: -117.81094885782845, yOrigin: 33.37395581476702


picture: subset117.png
picture_name: subset117
image_width: 640, image_height: 640
xOrigin: -117.81094885782845, yOrigin: 33.31646363658338


picture: subset12.png
picture_name: subset12
image_width: 640, image_height: 640
xOrigin: -118.4433628178486, yOrigin: 33.66141670568527


picture: subset13.png
picture_name: subset13
image_width: 640, image_height: 640
xOrigin: -118.4433628178486, yOrigin: 33.603924527501626


picture: subset14.png
picture_name: subset14
image_width: 640, image_height: 640
xOrigin: -118.4433628178486, yOrigin: 33.546432349317975


picture: subset15.png
picture_name: subset15
image_width: 640, image_height: 640
xOrigin: 

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass,LONpx_X2,LATpx_Y2
0,367682160,2021-03-07 01:50:17.500,33.31485,-118.393618,1.3,0.3,511.0,ANGELS GATE,IMO0000000,WDI2830,30.0,NaN,16.0,12.0,NaN,NaN,B,554,18


Number of ships founded (including black areas = 0 pixel value): 1
Number of ships founded (including black areas = 0 pixel value): 1
Number of ships founded after excluding black areas == 0 pixel value: 0

picture: subset19.png
picture_name: subset19
image_width: 640, image_height: 640
xOrigin: -118.38587063966494, yOrigin: 33.77640106205257


picture: subset2.png
picture_name: subset2
image_width: 640, image_height: 640
xOrigin: -118.50085499603225, yOrigin: 33.71890888386892


picture: subset20.png
picture_name: subset20
image_width: 640, image_height: 640
xOrigin: -118.38587063966494, yOrigin: 33.71890888386892


picture: subset21.png
picture_name: subset21
image_width: 640, image_height: 640
xOrigin: -118.38587063966494, yOrigin: 33.66141670568527


picture: subset22.png
picture_name: subset22
image_width: 640, image_height: 640
xOrigin: -118.38587063966494, yOrigin: 33.603924527501626


picture: subset23.png
picture_name: subset23
image_width: 640, image_height: 640
xOrigin: -118

...

Number of ships founded (including black areas = 0 pixel value): 19
Number of ships founded (including black areas = 0 pixel value): 19
Number of ships founded after excluding black areas == 0 pixel value: 0

picture: subset85.png
picture_name: subset85
image_width: 640, image_height: 640
xOrigin: -117.9834253923794, yOrigin: 33.603924527501626


picture: subset86.png
picture_name: subset86
image_width: 640, image_height: 640
xOrigin: -117.9834253923794, yOrigin: 33.546432349317975


picture: subset87.png
picture_name: subset87
image_width: 640, image_height: 640
xOrigin: -117.9834253923794, yOrigin: 33.488940171134324


picture: subset88.png
picture_name: subset88
image_width: 640, image_height: 640
xOrigin: -117.9834253923794, yOrigin: 33.43144799295067


picture: subset89.png
picture_name: subset89
image_width: 640, image_height: 640
xOrigin: -117.9834253923794, yOrigin: 33.37395581476702


picture: subset9.png
picture_name: subset9
image_width: 640, image_height: 640
xOrigin: -118.

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass,LONpx_X2,LATpx_Y2
0,338339572,2021-03-07 01:50:17.500,33.604160,-117.887670,0.1,360.0,511.0,TUNA MELT,IMO0000000,NaN,37.0,NaN,14.0,5.0,NaN,NaN,B,426,637
1,367163340,2021-03-07 01:50:17.500,33.610062,-117.908743,0.0,241.7,511.0,MEHETABEL,NaN,WDD5767,37.0,NaN,22.0,5.0,NaN,NaN,B,191,572
2,338328028,2021-03-07 01:50:17.500,33.623267,-117.894690,0.0,360.0,145.0,AQUATA,IMO0000000,NaN,37.0,NaN,18.0,5.0,NaN,NaN,B,348,425
3,367691350,2021-03-07 01:50:17.500,33.613080,-117.924402,0.0,293.6,511.0,ANDIAMO,IMO0000000,NaN,37.0,NaN,21.0,6.0,NaN,NaN,B,17,538
4,367678390,2021-03-07 01:50:17.500,33.613630,-117.924750,0.0,273.3,227.0,EL TIGRE,NaN,WDI2461,37.0,0.0,19.0,6.0,NaN,0.0,A,13,532


Number of ships founded (including black areas = 0 pixel value): 75
Number of ships founded (including black areas = 0 pixel value): 75
Number of ships founded after excluding black areas == 0 pixel value: 0

picture: subset94.png
picture_name: subset94
image_width: 640, image_height: 640
xOrigin: -117.92593321419575, yOrigin: 33.603924527501626
##############
SHIPS DETECTED
##############


,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass,LONpx_X2,LATpx_Y2
0,367740030,2021-03-07 01:50:17.500,33.603083,-117.898783,2.0,143.6,511.0,NEWPORT LEGACY,NaN,WDI8578,60.0,0.0,NaN,NaN,NaN,NaN,A,302,9
1,338339611,2021-03-07 01:50:17.500,33.601960,-117.897440,0.0,360.0,318.0,VAL AIR,IMO0000000,NaN,37.0,NaN,11.0,4.0,NaN,NaN,B,317,22
2,367001740,2021-03-07 01:50:17.500,33.603000,-117.899100,0.0,96.0,213.0,CATALINA FLYER,NaN,WTD8887,60.0,0.0,34.0,11.0,NaN,NaN,A,299,10


Number of ships founded (including black areas = 0 pixel value): 3
Number of ships founded (including black areas = 0 pixel value): 3
Number of ships founded after excluding black areas == 0 pixel value: 0

picture: subset95.png
picture_name: subset95
image_width: 640, image_height: 640
xOrigin: -117.92593321419575, yOrigin: 33.546432349317975


picture: subset96.png
picture_name: subset96
image_width: 640, image_height: 640
xOrigin: -117.92593321419575, yOrigin: 33.488940171134324


picture: subset97.png
picture_name: subset97
image_width: 640, image_height: 640
xOrigin: -117.92593321419575, yOrigin: 33.43144799295067


picture: subset98.png
picture_name: subset98
image_width: 640, image_height: 640
xOrigin: -117.92593321419575, yOrigin: 33.37395581476702


picture: subset99.png
picture_name: subset99
image_width: 640, image_height: 640
xOrigin: -117.92593321419575, yOrigin: 33.31646363658338


